[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/SD-CN-Animation-colab/blob/main/SD-CN-Animation-colab.ipynb)

In [ ]:
%cd /content
!git clone https://huggingface.co/camenduru/SD-CN-Animation
%cd /content/SD-CN-Animation

!pip install yt_dlp
from yt_dlp import YoutubeDL
def download_video(url):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':'/content/SD-CN-Animation/input.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(url)
    return f"/content/SD-CN-Animation/input.mp4"

In [ ]:
download_video("https://www.youtube.com/watch?v=EU3hIXXeiz4")

In [ ]:
%cd /content/SD-CN-Animation
!python compute_flow.py -i /content/SD-CN-Animation/input.mp4 -o /content/SD-CN-Animation/flow.h5 -v -W 720 -H 1280

In [ ]:
%cd /content/SD-CN-Animation
!python vid2vid.py

In [ ]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U

!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
%cd /content/stable-diffusion-webui

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/mistoonanime/resolve/main/mistoonAnime_v10.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o mistoonAnime_v10.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/grapefruit/resolve/main/grapefruit.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o mistoonAnime_v10.safetensors

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/stable-diffusion-webui/modules/shared.py
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --api